# <b>Theoreticl Questions<b>

**Q1:**
<br>
To evaluate how well our model performs at T1D classification, we need to have evaluation metrics that measures of its performances/accuracy. Which evaluation metric is more important to us: model accuracy or model performance? Give a simple example that illustrates your claim

**A1:**
<br>
The model performance would be a better evaluation metric in this case. Model accuracy is just the number of correct predictions made by the model devided by the total number of predictions - which would not be a good model in this case because we have a relatively small number of people in the population with the T1D. For example, if we use a classifier that classifies all patients as being healthy, we would get an accuracy of at least 99.67% (according to the assignment that states that up to 0.33% suffer from T1D), which is a very high accuracy, but this is a terrible classifier.

**Q2:**
<br>
T1D is often associated with other comorbidities such as a heart attack. You are asked to design a ML algorithm to predict which patients are going to suffer a heart attack. Relevant patient features for the algorithm may include blood pressure (BP), body-mass index (BMI), age (A), level of physical activity (P), and income (I). You should choose between two classifiers: the first uses only BP and BMI features and the other one uses all of the features available to you. Explain the pros and cons of each choice.

**A2:** 

**Classifier1**
<br>
Pros - Easy to visualize data, less weights to train.
<br>
Cons - Selected feature may not be the best features, the less features we have - the more missing information in one of the features might increase errors in the classification.
    
**Classifier2**
<br>
Pros - Contains a wider selection of features which might help get a better classification.
<br>
Cons - Longer training time, may contain redundant features.

**Q3:**
<br>
A histologist wants to use machine learning to tell the difference between pancreas biopsies that show signs of T1D and those that do not. She has already come up with dozens of measurements to take, such as color, size, uniformity and cell-count, but she isn’t sure which model to use. The biopsies are really similar, and it is difficult to distinguish them from the human eye, or by just looking at the features. Which of the following is better: logistic regression, linear SVM or nonlinear SVM? Explain your answer.

**A3:**
<br>
Since the data is difficult to distinguish by just looking at the features, logistic regression would not be useful for classification, nor would linear SVM, since the data is non-seperable, and therefore we cannot seperate the data using a linear line.
In this case, nonlinear SVM would be the most useful method, since it would transform our non-seperable data into a higher dimensional space, where the data might prove to be seperable.

**Q4:**
<br>
What are the differences between LR and linear SVM and what is the difference in the effect/concept of their hyper-parameters tuning?

**A4:** 
The main difference between LR and Linear SVM is that linear SVM find the "best" margin that seperates the classes, while LR does not, and can have different boundaries with different weights (the boundary is not "optimized").
Another differense is that LR uses a probobilistic approach, while SVM is deterministic.
The differnce in the concept of their hyper-parameters tuning is that LR uses a regularization (penalty) - for example, l1 or l2, and the C parameter which gives us the penalty strength. For linear SVM - we use the "kernel trick" in order to find the best hypter-parameters.

# <b>Coding Assignment<b>

In [1]:
# imports
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
#T1D data 
df_raw = pd.read_csv("HW2_data.csv")
df = df_raw.copy()

#preprocessing
#finding empty cells in dataframe
row,col = np.where(pd.isnull(df))
#converting Yes/No to 1/0
for i in range(2,16):
    df.iloc[:,i] = df.iloc[:,i].map({'Yes':1,'No':0})
df.iloc[:,16] = df.iloc[:,16].map({'Positive':1,'Negative':0})
#converting empty cell to NaN
for i,j in zip(row,col):
    df.iloc[i,j] = np.nan



In [4]:
# train test split
# feature columns
features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17]
X = df.iloc[:,features]
# T1D prediction
y = df.iloc[:,16]
X_train, x_test, Y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 10, stratify=y)

In [ ]:
# data visualization and exploration